In [18]:
import os
import time
import select
import subprocess
import pandas as pd

# check if csv file is present

if os.path.exists('/home/gnuradio/dump1090_sdrplus-master/aircraft_log.csv'):
    pass
else:
    print('not there')

### Initializing Lists ###

airplane_list = []

source = []



In [19]:
### MAIN LOOP ###

## we use a list of dicts to store all plane information ##

## Checks the end of csv file ##            
## (last line provides most up-to-date info about an aircraft) ##

with open("/home/gnuradio/dump1090_sdrplus-master/aircraft_log.csv") as file:
    for last_line in file:
        pass
    slice_list = (list(map(str.strip, last_line.split(","))))

## replace old info with new info ##
for i in airplane_list:
    if i['Hex'] == slice_list['Hex']:
        airplane_list.remove(i)

airplane_list.append(({'Hex': slice_list[0],
                        'Flight': slice_list[1],
                        'Altitude': slice_list[2],
                        'Speed': slice_list[3],
                        'Lat': slice_list[4],
                        'Lon': slice_list[5],
                        'Track': slice_list[6],
                        'Messages': slice_list[7],
                        'Seen': slice_list[8]}))
print(airplane_list)


## Uses the entire csv file ##

# source = []
# slice_dict = {'Hex': [],'Flight': [],'Altitude': [],'Speed': [],'Lat': [],'Lon': [],'Track': [],'Messages': [],'Seen': []}

# with open("/home/gnuradio/dump1090_sdrplus-master/aircraft_log.csv") as file:
#     f = file.readlines()
#     for i in f[1:]:
#         slice_list = list(map(str.strip, i.split(",")))
#         source.append({'Hex': slice_list[0],
#                        'Flight': slice_list[1],
#                        'Altitude': slice_list[2],
#                        'Speed': slice_list[3],
#                        'Lat': slice_list[4],
#                        'Lon': slice_list[5],
#                        'Track': slice_list[6],
#                        'Messages': slice_list[7],
#                        'Seen': slice_list[8]})
    


[{'Hex': 'a45586', 'Flight': 'DAL1530', 'Altitude': '7125', 'Speed': '259', 'Lat': '0.000000', 'Lon': '0.000000', 'Track': '85', 'Messages': '52', 'Seen': '1677185367'}]


In [20]:
## IMPORTING LIBRARY
import requests
import json
import pandas as pd

## IMPORTING PLOTTING LIBRARIES
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider,STAMEN_TERRAIN
from bokeh.models import HoverTool,LabelSet,ColumnDataSource
import numpy as np

## AREA EXTENT COORDINATE WGS4 (SET THIS TO WEST LA)
lon_min,lat_min= -119,33.0
lon_max,lat_max= -118.0,34.4



In [21]:
## FUNCTION TO CONVERT GCS WGS84 TO WEB MERCATOR POINT

def wgs84_web_mercator_point(lon,lat):
    k = 6378137
    x= lon * (k * np.pi/180.0)
    y= np.log(np.tan((90 + lat) * np.pi/360.0)) * k
    return x,y

#DATA FRAME
def wgs84_to_web_mercator(df, lon="Lon", lat="Lat"):
    k = 6378137
    df["x"] = int(float(df[lon])) * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + int(float(df[lat]))) * np.pi/360.0)) * k
    return df

In [22]:
## CREATE EMPTY PANDAS DATAFRAME W/ COLUMNS ##
col_name = ['Hex','Flight','Altitude','Speed','Lat','Lon','Track','Messages','Seen']
flight_df = pd.DataFrame(columns = col_name )


## COORDINATE CONVERSION

xy_min=wgs84_web_mercator_point(lon_min,lat_min)
xy_max=wgs84_web_mercator_point(lon_max,lat_max)

flights = []

for i in airplane_list:
    
    wgs84_to_web_mercator(i)

## populate pandas df ##
flight_df = pd.DataFrame(airplane_list)
flight_df=flight_df.fillna('No Data') #replace NAN with No Data
print(flight_df)


#flight_df['rot_angle']=flight_df['true_track']*-1 #Rotation angle
icon_url='https://img1.pnghut.com/4/14/18/52nsQBCssu/paper-monochrome-cursor-logo-triangle.jpg' #Icon url
flight_df['url']=icon_url

      Hex   Flight Altitude Speed       Lat       Lon Track Messages  \
0  a45586  DAL1530     7125   259  0.000000  0.000000    85       52   

         Seen    x             y  
0  1677185367  0.0 -7.081155e-10  


In [23]:
def modify_doc(doc):
    ## FIGURE SETTING
    x_range,y_range=([xy_min[0],xy_max[0]], [xy_min[1],xy_max[1]])
    p=figure(x_range=x_range,y_range=y_range,x_axis_type='mercator',y_axis_type='mercator',sizing_mode='scale_width',height=300)

    ## PLOT BASEMAP AND AIRPLANE POINTS
    flight_source=ColumnDataSource(flight_df)
    tile_prov=get_provider('CARTODBPOSITRON')
    p.add_tile(tile_prov,level='image')
    p.image_url(url='url', x='x', y='y',source=flight_source,anchor='center',angle_units='deg',h_units='screen',w_units='screen',w=40,h=40)
    p.circle('x','y',source=flight_source,fill_color='red',hover_color='yellow',size=10,fill_alpha=0.8,line_width=0)

    ## HOVER INFORMATION AND LABEL
    my_hover=HoverTool()
    my_hover.tooltips=[('ICAO','@Hex'),('Speed(m/s)','@Speed'),('Altitude(m)','@Altitude')]
    labels = LabelSet(x='x', y='y', text='callsign', level='glyph',
                x_offset=5, y_offset=5, source=flight_source,background_fill_color='white',text_font_size="8pt")
    p.add_tools(my_hover)
    p.add_layout(labels)

    doc.add_root(p)
    






In [24]:
start = time.time()
show(p)
print(time.time() - start)

5.252490043640137
